In [1]:
from langame_client import LangameClient
from pprint import pprint
from firebase_admin import firestore
import requests
from bs4 import BeautifulSoup
import json
import openai
from pathlib import Path
import pandas as pd
import datetime
import markdown
c = LangameClient()
READWISE_TOKEN="grab me here https://readwise.io/access_token"

In [3]:
out_file_name = "data/brain-0.0.1.jsonl"
count_file = 0
for p in Path("../brain").rglob("*.[mM][dD]"):
  count_file += 1
  with open(p) as f:
    lines = "".join(f.readlines())
    with open(out_file_name, "a+") as outfile:
      json.dump({
          "prompt": f"{str(p.stem)}:",
          "completion": f" {lines}\n###\n",
      }, outfile)
      outfile.write('\n')
print("found", count_file, "files")

found 657 files


In [2]:
def get_all_highlights():
  highlights = []
  page = 1
  url = f"https://readwise.io/api/v2/highlights/"
  while url:
    response = requests.get(
        url=url,
        headers={"Authorization": f"Token {READWISE_TOKEN}"},
    ).json()
    if "results" in response:
      highlights = highlights + response["results"]

    url = response["next"] if "next" in response else None
    page += 1
  return highlights
highlights = get_all_highlights()

In [4]:
out_file_name = "data/readwise-0.0.1.jsonl"
count_file = 0
df = pd.DataFrame(highlights)
with open(out_file_name, "a+") as outfile:
  for i, v in df.iterrows():
    prompt = ','.join([e['name'] for e in v['tags']])
    if prompt: prompt += ":"
    json.dump({
        "prompt": f"{prompt}",
        "completion": f" {v.text}\n###\n",
    }, outfile)
    outfile.write('\n')

In [7]:
memes = []
for e in c._firestore_client.collection('memes').stream():
    memes.append((e.id, e.to_dict()))

In [9]:
memes[0]

('0FNnecFpEToBI1mwAIJF',
 {'topics': ['health'],
  'createdAt': DatetimeWithNanoseconds(2021, 7, 6, 9, 1, 3, 85000, tzinfo=<UTC>),
  'content': 'Do you have any type of disability or chronic illness?',
  'translated': {'de': 'Haben Sie eine Behinderung oder eine chronische Krankheit?',
   'es': '¿Tiene algún tipo de discapacidad o enfermedad crónica?',
   'fr': 'Avez-vous un type de handicap ou de maladie chronique?'}})

In [12]:
file_name = 'data/memes-0.0.1.jsonl'
with open(file_name, 'a+') as outfile:
    for meme in memes:
        prompt = ','.join([e for e in meme[1]['topics']])
        if prompt: prompt += ":"
        json.dump({
            "prompt": f"{prompt}",
            "completion": f" {meme[1]['content']}\n###\n"
        }, outfile)
        outfile.write('\n')

In [3]:
file_name = 'data/brain-0.0.1-readwise-0.0.1-memes-0.0.1.jsonl'

In [13]:
data = data2 = data3 = ""
  
with open('data/brain-0.0.1.jsonl') as fp:
    data = fp.read()
  
with open('data/readwise-0.0.1.jsonl') as fp:
    data2 = fp.read()

with open('data/memes-0.0.1.jsonl') as fp:
    data3 = fp.read()
  
data += data2
data += data3

with open (file_name, 'w') as fp:
    fp.write(data)

In [16]:
!openai tools fine_tunes.prepare_data -f $file_name

Analyzing...

- Your file contains 3163 prompt-completion pairs
- There are 8 duplicated prompt-completion pairs. These are rows: [733, 1249, 1263, 1319, 2064, 3065, 3069, 3089]
- There are 6 examples that are very long. These are rows: [27, 31, 41, 45, 46, 338]
For conditional generation, and for classification the examples shouldn't be longer than 2048 tokens.
- Your data does not contain a common separator at the end of your prompts. Having a separator string appended to the end of the prompt makes it clearer to the fine-tuned model where the completion should begin. See https://beta.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more detail and examples. If you intend to do open-ended generation, then you should leave the prompts empty
- All completions end with suffix `\n###\n`

Based on the analysis we will perform the following actions:
- [Recommended] Remove 8 duplicate rows [Y/n]: ^C



In [7]:
f = openai.File.create(
  file=open(file_name),
  purpose="fine-tune"
)
f_t = openai.FineTune.create(
    training_file=f["id"],
    model="curie"
)

Upload progress: 100%|██████████| 1.30M/1.30M [00:01<00:00, 1.28Mit/s]


In [16]:
model = openai.FineTune.list()['data'][-1]
model

In [17]:
p = {}
p["type"] = file_name
p["template"] = """This is a conversation starter about [TOPIC].
Conversation starter:"""
p["parameters"] = {
    "model": model["fine_tuned_model"],
    "temperature": 0.7,
    "maxTokens": 150,
    "topP": 1,
    "frequencyPenalty": 0.1,
    "presencePenalty": 0.1,
    "stop": ["\n###\n", "###", "\n", "Conversation starter", "starter"],
}
p_col = c._firestore_client.collection("prompts")
new_prompt_ref = p_col.add(p)

In [34]:
openai.Completion.create(
  model=model["fine_tuned_model"],
  prompt="",
  max_tokens=300,
  stop=["\n###\n", "###"],
)

<OpenAIObject text_completion id=cmpl-3bb9PbAkCra9NSEJrOTKDhDxi4yCP at 0x13f15c770> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "While most of humanity is focused on avoiding harm, a small but very vocal minority has decided that the best way to avoid harm is to actively seeks it out. Called \u201cthe optimists\u201d by the media, this group includes anarchists, Daredevil bloggers, thrill-seekers, and other \u2013 sometimes bizarre \u2013 personalities."
    }
  ],
  "created": 1630089955,
  "id": "cmpl-3bb9PbAkCra9NSEJrOTKDhDxi4yCP",
  "model": "curie:ft-louis-personal-2021-08-27-16-57-14",
  "object": "text_completion"
}